<a href="https://colab.research.google.com/github/deniskapel/ner-dialogues-hackathon/blob/master/Baseline_Flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flair

     |████████████████████████████████| 450kB 8.2MB/s 
     |████████████████████████████████| 71kB 2.9MB/s 
     |████████████████████████████████| 1.3MB 14.8MB/s 
     |████████████████████████████████| 983kB 46.4MB/s 
     |████████████████████████████████| 1.1MB 31.9MB/s 
     |████████████████████████████████| 798kB 33.7MB/s 
     |████████████████████████████████| 19.7MB 6.1MB/s 
     |████████████████████████████████| 2.9MB 24.8MB/s 
     |████████████████████████████████| 890kB 21.3MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.7.0-cp36-none-any.whl size=14376 sha256=0f62e7ee50df56c84c2f064e94f300f9ea0790cd2cbfb1a424b7c59980980486
  Stored in directory: /root/.cache/pip/wheels/cf/c6/4f/2c64a43f041415eb8b8740bd80e15e92f0d46c5e464d8e4b9b
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45613 sha256=8a47d93a0ddc8ae7885b49ff9ddd5d536a5e2a63cfab2580aa58223c21b46058
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34

In [ ]:
# Uncomment if you need to download data
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
!wget https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/test.conll

--2021-02-09 15:37:22--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/train.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125059 (122K) [text/plain]
Saving to: ‘train.conll’

train.conll         100%[===================>] 122.13K  --.-KB/s    in 0.01s   

2021-02-09 15:37:23 (8.04 MB/s) - ‘train.conll’ saved [125059/125059]

--2021-02-09 15:37:23--  https://raw.githubusercontent.com/Rexhaif/ner-dialogues-hackathon/master/data/dev.conll
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14109 (14K) [text

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, CharacterEmbeddings, FlairEmbeddings

In [ ]:
import flair
import torch
# uncomment for cpu-only training
# flair.device = torch.device('cpu')

In [ ]:
# 1. get the corpus
corpus: Corpus = ColumnCorpus(
    "./", column_format={0:'text', 1:'ner'},
    train_file='train.conll', dev_file='dev.conll', test_file='test.conll',
)
print(corpus)

2021-02-09 15:39:35,653 Reading data from .
2021-02-09 15:39:35,658 Train: train.conll
2021-02-09 15:39:35,659 Dev: dev.conll
2021-02-09 15:39:35,661 Test: test.conll
Corpus: 1966 train + 219 dev + 547 test sentences


In [ ]:
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary)

# 4. initialize embeddings
embedding_types = [
    # standard word embeddings
    WordEmbeddings('glove'),
    CharacterEmbeddings()
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

Dictionary with 14 tags: <unk>, O, B-SINGER, I-SINGER, B-SONG, B-FILM, I-SONG, I-FILM, B-COMPOSER, I-COMPOSER, B-BOOK, I-BOOK, <START>, <STOP>
2021-02-09 16:08:05,589 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmp76fsa7ae


100%|██████████| 160000128/160000128 [00:09<00:00, 16483738.92B/s]

2021-02-09 16:08:15,775 copying /tmp/tmp76fsa7ae to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2021-02-09 16:08:16,051 removing temp file /tmp/tmp76fsa7ae
2021-02-09 16:08:16,726 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpab7gfsl0


100%|██████████| 21494764/21494764 [00:02<00:00, 10152404.42B/s]

2021-02-09 16:08:19,313 copying /tmp/tmpab7gfsl0 to cache at /root/.flair/embeddings/glove.gensim
2021-02-09 16:08:19,339 removing temp file /tmp/tmpab7gfsl0


In [ ]:
# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

# 7. start training
history = trainer.train(
    './models/baseline-charembeddings',
    learning_rate=0.1,
    mini_batch_size=32,
    max_epochs=20
)

2021-02-09 16:08:21,023 ----------------------------------------------------------------------------------------------------
2021-02-09 16:08:21,027 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
    (list_embedding_1): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=150, out_features=150, bias=True)
  (rnn): LSTM(150, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=14, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2021-02-09 16:08:21,028 ----------------------------------------------------------------------------------------------------
2021-02-09 16:08:21,030 Corpus: "Corpus: 1966 train + 219 dev + 547 test sentences"
2021-02-09 16:08:21,033 ----------------------------